# Top Pages Search Performance

Use this notebook to identify which pages are carrying search performance and where page-level optimization is likely to pay off.

In [ ]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../..")

import plotly.express as px

import lifeline_theme
from lla_data import config
from lla_data.bq import build_date_params, default_query_window, get_client, run_query

lifeline_theme.inject_fonts()

client = get_client()
window = default_query_window(config.DEFAULT_DAYS_BACK)
TOP_N = config.DEFAULT_TOP_N

In [ ]:
query = f"""
SELECT
  page_path,
  SUM(gsc_clicks) AS clicks,
  SUM(gsc_impressions) AS impressions,
  SAFE_DIVIDE(SUM(gsc_clicks), NULLIF(SUM(gsc_impressions), 0)) AS ctr,
  SAFE_DIVIDE(SUM(gsc_avg_position * gsc_impressions), NULLIF(SUM(gsc_impressions), 0)) AS avg_position,
  SUM(organic_sessions) AS organic_sessions
FROM `{config.PROJECT_ID}.{config.SEARCHCONSOLE_DATASET}.seo_page_daily`
WHERE report_date BETWEEN DATE(@start_date) AND DATE(@end_date)
GROUP BY page_path
HAVING impressions > 0
ORDER BY clicks DESC
LIMIT {TOP_N}
"""

df_pages = run_query(client, query, params=build_date_params(window))
df_pages.head()

In [ ]:
fig_clicks = px.bar(
    df_pages.sort_values("clicks", ascending=True),
    x="clicks",
    y="page_path",
    orientation="h",
    template="lifeline",
    title=f"Top {TOP_N} Pages by Search Clicks",
)
fig_clicks.update_layout(height=700, margin={"l": 260})
lifeline_theme.add_lifeline_logo(fig_clicks)
fig_clicks.show()

fig_ctr = px.scatter(
    df_pages,
    x="avg_position",
    y="ctr",
    size="impressions",
    hover_name="page_path",
    template="lifeline",
    title="CTR vs Average Position (bubble size = impressions)",
)
fig_ctr.update_yaxes(tickformat=".0%")
lifeline_theme.add_lifeline_logo(fig_ctr)
fig_ctr.show()